In [47]:
%load_ext autoreload
%autoreload 2

import diveslowlearnfast as dlf
import numpy as np

from diveslowlearnfast.datasets import Diving48Dataset
from diveslowlearnfast.datasets.diving48 import extend_under_represented_classes
from diveslowlearnfast.config import Config

cfg = Config()
cfg.DATA.DATASET_PATH = '/Users/youritomassen/Projects/xai/data/Diving48/'
cfg.DATA.NUM_FRAMES = 4

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
diving48 = Diving48Dataset(
    cfg.DATA.DATASET_PATH,
    cfg.DATA.NUM_FRAMES,
    cfg.SLOWFAST.ALPHA,
    transform_fn=dlf.get_test_transform(cfg)
)

In [8]:
idx = 0
x, y, *_ = diving48[idx]

x.shape, diving48.get_label(y)

(torch.Size([3, 4, 256, 256]), ['Back', '15som', '05Twis', 'FREE'])

In [20]:
data = diving48.data

labels = np.array(list(map(lambda x: x['label'], data)))

unique, counts = np.unique(labels, return_counts=True)
idx = np.argmax(counts)
max_count = counts[idx]
unique[idx], max_count



(26, 1053)

In [38]:
new_data = []
start = 0
for count in counts:
    print(start)
    if count >= max_count:
        new_data.extend(data[start:count])
        start += count
        continue
    # print(max_count // count)
    a = data[start:start+count] * (max_count // count)
    new_data.extend(a)

    start += count

0
222
331
380
639
672
1139
1299
2200
2624
2836
2910
3054
3475
3519
3621
4212
4260
4609
4677
5281
5429
6118
6473
6530
7014
7060
8113
8363
8689
8920
9603
9664
10232
10803
11795
12141
12210
12301
12346
12394
12554
12684
13090
13608
14167
14983


In [39]:
len(data), len(new_data)

(15027, 42182)

In [40]:
labels = np.array(list(map(lambda x: x['label'], new_data)))
unique, counts = np.unique(labels, return_counts=True)

counts[:5]

array([ 888,  981, 1029, 1036, 1023])

In [45]:
new_data = extend_under_represented_classes(diving48.data)
labels = np.array(list(map(lambda x: x['label'], new_data)))
unique, counts = np.unique(labels, return_counts=True)

counts[:5]

array([ 888,  981, 1029, 1036, 1023])

In [52]:
cfg.DATA.EXTEND_CLASSES = True
diving48 = Diving48Dataset(
    cfg.DATA.DATASET_PATH,
    cfg.DATA.NUM_FRAMES,
    cfg.SLOWFAST.ALPHA,
    extend_classes=cfg.DATA.EXTEND_CLASSES,
    transform_fn=dlf.get_test_transform(cfg)
)

len(diving48.data)

43235